In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data fetching

In [68]:
csv1 = pd.read_csv('Data/Books.csv')
csv2 = pd.read_csv('Data/BX-Books.csv', sep =";",encoding="latin" , on_bad_lines='skip')

csv3 = pd.read_csv('Data/Users.csv')
csv4 = pd.read_csv('Data/BX-Users.csv', sep =";",encoding="latin" , on_bad_lines='skip')

csv5 = pd.read_csv('Data/Ratings.csv')
csv6 = pd.read_csv('Data/BX-Book-Ratings.csv', sep =";",encoding="latin" , on_bad_lines='skip')

C:\Users\PC PAL\AppData\Local\Temp\ipykernel_5616\3947458865.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  csv1 = pd.read_csv('Data/Books.csv')
C:\Users\PC PAL\AppData\Local\Temp\ipykernel_5616\3947458865.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  csv2 = pd.read_csv('Data/BX-Books.csv', sep =";",encoding="latin" , on_bad_lines='skip')


In [69]:
combined_df = pd.concat([csv1, csv2], ignore_index=True)

# Drop duplicate rows based on the "ISBN" column
unique_df = combined_df.drop_duplicates(subset='ISBN')

# Save the unique rows to a new CSV file
unique_df.to_csv('Data/All_Books.csv', index=False)

In [70]:
books = pd.read_csv('Data/All_Books.csv')  
books.shape

C:\Users\PC PAL\AppData\Local\Temp\ipykernel_5616\2865203439.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Data/All_Books.csv')


(271360, 8)

In [71]:
combined_df = pd.concat([csv3, csv4], ignore_index=True)

# Drop duplicate rows based on the "ISBN" column
unique_df = combined_df.drop_duplicates(subset='User-ID')

# Save the unique rows to a new CSV file
unique_df.to_csv('Data/All_users.csv', index=False)

In [72]:
users= pd.read_csv('Data/All_users.csv')
users.shape

(278858, 3)

In [73]:
combined_df = pd.concat([csv5, csv6], ignore_index=True)

# Drop duplicate rows based on the "ISBN" column
unique_df = combined_df.drop_duplicates(subset=['User-ID','ISBN'])

# Save the unique rows to a new CSV file
unique_df.to_csv('Data/All_ratings.csv', index=False)

In [74]:
ratings = pd.read_csv('Data/All_ratings.csv')
ratings.shape

(1149780, 3)

## Pre processing

In [75]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [76]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [77]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [78]:
books.rename(columns ={
    'Book-Title':'Title',
    'Book-Author':'Author',
    'Year-Of-Publication':'Year',
    'Image-URL-L':'Image'
}, inplace=True)        


In [79]:
users.columns

Index(['User-ID', 'Location', 'Age'], dtype='object')

In [80]:
users.rename(columns ={
    'User-ID':'User',
}, inplace=True)

In [81]:
ratings.columns

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [82]:
ratings.rename(columns ={
    'User-ID':'User',
    'Book-Rating':'Rating'
}, inplace=True)    
    

In [83]:
print(books.columns)
print(users.columns)
print(ratings.columns)

Index(['ISBN', 'Title', 'Author', 'Year', 'Publisher', 'Image'], dtype='object')
Index(['User', 'Location', 'Age'], dtype='object')
Index(['User', 'ISBN', 'Rating'], dtype='object')


In [84]:
ratings['User'].value_counts()

User
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [85]:
x = ratings['User'].value_counts() > 200

In [86]:
x[x].shape

(899,)

In [87]:
y = x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='User', length=899)

In [88]:
ratings = ratings[ratings['User'].isin(y)]

In [106]:
ratings_with_books = ratings.merge(books ,on ='ISBN')

In [107]:
number_rating = ratings_with_books.groupby('Title')['Rating'].count().reset_index()

In [108]:
number_rating.rename(columns={'Rating':'num_of_rating'},inplace=True)

In [109]:
final_rating = ratings_with_books.merge(number_rating, on='Title')

In [110]:
# Lets take those books which got at least 50 rating of user

final_rating = final_rating[final_rating['num_of_rating'] >= 50]

In [112]:
# lets drop the duplicates
final_rating.drop_duplicates(['User','Title'],inplace=True)

In [113]:
# Lets create a pivot table
book_pivot = final_rating.pivot_table(columns='User', index='Title', values= 'Rating')

In [114]:

book_pivot.shape

(742, 888)

In [115]:

book_pivot.fillna(0, inplace=True)

In [116]:

book_pivot

User,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Train the model

In [117]:

from scipy.sparse import csr_matrix

In [118]:
book_sparse = csr_matrix(book_pivot)

In [119]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [120]:

# Now import our clustering algoritm which is Nearest Neighbors this is an unsupervised ml algo
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [121]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [122]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )

In [123]:
distance

array([[ 0.        , 67.75691847, 68.05145112, 72.277244  , 75.81556568,
        76.30203143]])

In [124]:
suggestion

array([[237, 238, 240, 241, 184, 536]], dtype=int64)

In [125]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='Title')


In [126]:

#keeping books name
book_names = book_pivot.index

In [128]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_names,open('artifacts/book_names.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))

## Testing

In [129]:


def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in range(len(suggestion)):
            books = book_pivot.index[suggestion[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)

In [130]:
book_name = "Harry Potter and the Chamber of Secrets (Book 2)"
recommend_book(book_name)

You searched 'Harry Potter and the Chamber of Secrets (Book 2)'

The suggestion books are: 

Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall
